<a href="https://colab.research.google.com/github/hermesvillarreal/CodigosDeInteres/blob/main/Solucion_Archivos_Base_Al_Server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

```
Contexto:
Tengo una carpeta con archivos base csv a los cuales debo:
* recibir por parametro 2 argumentos, 1 obligatorio y 1 opcional
* modificar el nombre del archivo
* revisar su contenido
* si no existe la carpeta de salida, crearla
* copiarlos a otra carpeta 
* comprimir en un zip la carpeta
* enviar por sftp el zip a un servidor
```
```
Links:
https://docs.python.org/es/3/library/datetime.html
https://docs.python.org/es/3/library/sys.html
https://docs.python.org/es/3/library/os.html
https://docs.python.org/es/3/library/shutil.html
https://docs.python.org/es/3/library/subprocess.html
https://docs.python.org/es/3/library/csv.html
```

importar las librerias a utilizar

In [ ]:
import datetime #Importar módulo de fecha y hora
import sys
import os
import shutil
from subprocess import check_output
import csv

establecer variables y lectura de los parametros

In [ ]:
### establecer ruta y nombre de carpeta de salida
rootdir = os.getcwd() 

today = datetime.date.today () #Obtener la fecha de hoy
yesterday = today - datetime.timedelta(days=1) #Obtener la fecha de ayer
tomorrow = today + datetime.timedelta(days=1) # Obtenga la fecha de mañana

dateparam = sys.argv[1]
print (dateparam)

match dateparam:
    case 'today':
        datefilename = today.strftime("%Y%m%d")
    case 'yesterday':
        datefilename = yesterday.strftime("%Y%m%d")
    case 'tomorrow':
        datefilename = tomorrow.strftime("%Y%m%d")
    case 'day':
        datefilename = sys.argv[2]
    case _:
        x = input ('Error en argumento. Se espera (today, yesterday, tomorrow, day (yyyymmdd))')
        exit()

FCT_DT_KEY = datefilename
FCT_DT = "\"" + datefilename[4:6] + "/" + datefilename[6:8] + "/" + datefilename[0:4] + "\""

print (datefilename, FCT_DT)

crear subcarpeta de ser necesario 

In [ ]:
inputdir = rootdir[0:-9] + 'InputFiles\\' + datefilename 
os.makedirs(inputdir, exist_ok=True)

tratamiento de los archivos csv en la carpeta base, 
* Si el archivo es nombrado con *CustomerGroup*.csv: se debe cambiar la columna FCT_DT_KEY y FCT_DT por la fecha incluida en el nombre de esta ejecución
* Si el archivo es cualquier otro csv: se copia en el destino, cambiando su nombre
* cualquier otro archivo es ignorado

In [ ]:
with os.scandir(rootdir) as ficheros:
    for fichero in ficheros:
        if fichero.name[4:17] == 'CustomerGroup' and fichero.name[-4:] == '.csv':
            with open(fichero.name) as csvfile:
                encreader = csv.DictReader(csvfile, delimiter='|', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                
                for encrow in encreader:
                    encabezado = encrow.keys()
            
            fields = list(encabezado)

            with open(inputdir + '\\' + fichero.name[0:-12] + datefilename + '.csv', 'w', newline='') as outcsv:
                writer = csv.writer(outcsv, delimiter='|', quotechar='\'', quoting=csv.QUOTE_MINIMAL)
                writer.writerow(fields)

                with open(fichero.name) as incsv:
                    reader = csv.DictReader(incsv, delimiter='|', quotechar='\'', quoting=csv.QUOTE_MINIMAL)
                    for row in reader:
                        row['FCT_DT_KEY'] = FCT_DT_KEY
                        row['FCT_DT'] = FCT_DT
                        writer.writerow(list(row.values()))
        if fichero.name[4:17] != 'CustomerGroup' and fichero.name[-4:] == '.csv': 
            shutil.copy(fichero.name, inputdir + '\\' + fichero.name[0:-12] + datefilename + '.csv')

print ('Archivos copiados en: ' + inputdir)

Se comprime la nueva carpeta

In [ ]:
format = "zip"
zipdir = rootdir[0:-9] + 'InputFiles'
zipname = zipdir + '\\' + datefilename 

shutil.make_archive(zipname, format, zipdir, datefilename)

print ('Zip creado: ' + zipname + '.' + format )

Se envia el zip a la ruta del ftp

localremote.txt
```
open sftp://user_ftp:pass_ftp@ip_server_ftp/ -hostkey="ssh-ed25519 255 67FRBlf84TIT4Se8g="
cd /planos
option transfer binary
put "%1%"
close
exit
```

In [ ]:
sftp = "\"C:\Program Files (x86)\WinSCP\WinSCP.com\""

script = rootdir + "\\local2remote.txt"
param = zipname + "." + format

comando = sftp + " /script=" + script + " /parameter " + param 
check_output(comando, shell=True)

print ('Enviado al servidor: ' + comando )



Se termina la ejecución

In [ ]:
x = input ('done!')